In [8]:
# 네이버 영화평 감성분석
import numpy as np
import pandas as pd

In [4]:
# 네이버 영화리뷰 데이터'로 검색
train_df = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', sep = '\t')
test_df = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt', sep = '\t')
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [6]:
train_df.shape, test_df.shape

((150000, 3), (50000, 3))

In [11]:
# 중복 여부 확인
train_df.document.nunique()

146182

In [13]:
# 중복 데이터 배제
train_df.drop_duplicates(subset = ['document'], inplace = True)
train_df.shape

(146183, 3)

In [15]:
# Null 데이터가 있는지 확인
train_df.isnull().sum()

id          0
document    1
label       0
dtype: int64

In [17]:
# Null 데이터 제거
train_df.dropna(how = 'any', inplace = True)
train_df.shape

(146182, 3)

In [19]:
# 긍정, 부정 레이블의 분포
train_df.label.value_counts()

0    73342
1    72840
Name: label, dtype: int64

In [20]:
# 중복 여부 확인
test_df.document.nunique()
print(test_df.document.nunique())
# 중복 데이터 배제
test_df.drop_duplicates(subset = ['document'], inplace = True)
print(test_df.shape)
# Null 데이터가 있는지 확인
test_df.isnull().sum()
print(test_df.isnull().sum())
# Null 데이터 제거
test_df.dropna(how = 'any', inplace = True)
print(test_df.shape)
# 긍정, 부정 레이블의 분포
test_df.label.value_counts()
print(test_df.label.value_counts())

49157
(49158, 3)
id          0
document    1
label       0
dtype: int64
(49157, 3)
1    24711
0    24446
Name: label, dtype: int64


In [22]:
# 텍스트 전처리
# 한글과 공백 이외는 제거
train_df['document'] = train_df.document.str.replace('[^ㄱ-하-ㅣ가-힣 ]', '')
train_df.head(3)

<ipython-input-22-35ff01a0f625>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  train_df['document'] = train_df.document.str.replace('[^ㄱ-하-ㅣ가-힣 ]', '')


,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [25]:
train_df['document'].replace('', np.nan, inplace = True)
train_df.document.isnull().sum()
print(train_df.document.isnull().sum())
train_df.dropna(how = 'any', inplace = True)
train_df.shape

363


(145819, 3)

In [28]:
test_df['document'] = test_df.document.str.replace('[^ㄱ-하-ㅣ가-힣 ]', '')
print(test_df.head(3))
test_df['document'].replace('', np.nan, inplace = True)
test_df.document.isnull().sum()
print(test_df.document.isnull().sum())
test_df.dropna(how = 'any', inplace = True)
test_df.shape

        id                          document  label
0  6270596                               굳 ㅋ      1
2  8544678  뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아      0
3  6825595         지루하지는 않은데 완전 막장임 돈주고 보기에는      0
0


<ipython-input-28-c4c7bf4a208f>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  test_df['document'] = test_df.document.str.replace('[^ㄱ-하-ㅣ가-힣 ]', '')


(49000, 3)

In [29]:
# 한글 처리
from konlpy.tag import Okt
okt = Okt()

In [40]:
df = pd.read_csv('C:\\Workspace\\python\\Data_Science\\dataA\\한국어불용어100.txt', encoding ='UTF-8', sep = '\s+', header = None)
df

,0,1,2
0,이,VCP,0.018280
1,있,VA,0.011699
2,하,VV,0.009774
3,것,NNB,0.009733
4,들,XSN,0.006898
...,...,...,...
95,원,NNB,0.000492
96,잘,MAG,0.000491
97,통하,VV,0.000487
98,소리,NNG,0.000486


In [45]:
# 품사를 무시하고 중복 배제
stopwords = set(df[0])
len(stopwords)

94

In [48]:
text = '교도소 이야기구먼 솔직히 재미는 없다평점 조정   '
okt.morphs(text, stem = True)

['교도소', '이야기', '구먼', '솔직하다', '재미', '는', '없다', '평점', '조정']

In [56]:
' '.join(okt.morphs(text, stem = True))


In [52]:
from tqdm.notebook import tqdm
str_train = []
for sentence in tqdm(train_df.document):
    morphs = okt.morphs(sentence.strip(), stem = True)
    temp_str = ' '.join(word for word in morphs if word not in stopwords)
    str_train.append(temp_str)

<ipython-input-52-a8ab04f85c82>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for sentence in tqdm_notebook(train_df.document):


  0%|          | 0/145819 [00:00<?, ?it/s]

In [62]:
str_train[:5]
len(str_train)

145819

In [57]:
str_test = []
for sentence in tqdm(test_df.document):
    morphs = okt.morphs(sentence.strip(), stem = True)
    temp_str = ' '.join(word for word in morphs if word not in stopwords)
    str_test.append(temp_str)

  0%|          | 0/49000 [00:00<?, ?it/s]

In [63]:
str_test[:5]
len(str_test)

49000

In [71]:
# Feature 변환
# CounterVectorizer
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer()

In [72]:
cvect.fit(str_train)
X_train = cvect.transform(str_train)
X_test = cvect.transform(str_test)
X_train.shape, X_test.shape

((145819, 42160), (49000, 42160))

In [73]:
# 모델 학습 / 예측 / 평가
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [76]:
y_train = train_df.label.values     # Numpy array
y_test = test_df.label.values
lr = LogisticRegression(max_iter=500)
lr.fit(X_train, y_train)
pred = lr.predict(X_test)
accuracy_score(y_test, pred)

0.8257755102040817